In [99]:
# 필요 라이브러리 호출
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from IPython.display import display
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'
from tqdm import tqdm

### 1. 안전지수 '심각' 지역 Elapsed Time 구하기

In [100]:
# 안전지수 '심각' 지역 데이터 호출
df_worst = pd.read_csv('./final_worst.csv')
df_worst = df_worst[['gid', 'geometry']]

In [101]:
df_worst.head()

,gid,geometry
0,다사50ab19ab,MULTIPOLYGON (((126.9388668024126 37.270833395...
1,다사51ab19aa,MULTIPOLYGON (((126.9501617690346 37.268632917...
2,다사52aa17bb,MULTIPOLYGON (((126.958701116179 37.2574048695...
3,다사55ab23bb,MULTIPOLYGON (((126.9949889452437 37.311648620...
4,다사56ab22bb,MULTIPOLYGON (((127.0063326534918 37.302682614...


In [102]:
worst_gid = df_worst['gid'].tolist()

In [103]:
len(worst_gid)

39

In [104]:
# 수원시 신고이력 데이터 호출
df_rep = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
df_rep.head()

,case_type,rec_date,rec_time,arrival_date,gid,name,psname
0,비상벨,20200101,183945',NaN,다사56aa19ab,고등파출소,수원서부경찰서
1,기타_타기관,20200101,012615',20200101014135.0,다사58ab18ba,인계파출소,수원남부경찰서
2,폭력,20200101,005151',20200101005314.0,다사56bb21aa,화서문지구대,수원중부경찰서
3,소음,20200101,012608',20200101013131.0,다사57aa17ba,세류지구대,수원남부경찰서
4,분실습득,20200101,014719',NaN,다사59bb21aa,광교지구대,수원남부경찰서


In [105]:
df_rep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223764 entries, 0 to 223763
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   case_type     223764 non-null  object
 1   rec_date      223764 non-null  int64 
 2   rec_time      223764 non-null  object
 3   arrival_date  199358 non-null  object
 4   gid           223764 non-null  object
 5   name          223764 non-null  object
 6   psname        223764 non-null  object
dtypes: int64(1), object(6)
memory usage: 12.0+ MB


In [106]:
df_rep['arrival_date'].isnull().sum()

24406

In [107]:
df_rep.dropna(subset=['arrival_date'], inplace=True)

In [108]:
df_rep['arrival_date'].isnull().sum()

0

In [109]:
df_rep.head()

,case_type,rec_date,rec_time,arrival_date,gid,name,psname
1,기타_타기관,20200101,012615',20200101014135.0,다사58ab18ba,인계파출소,수원남부경찰서
2,폭력,20200101,005151',20200101005314.0,다사56bb21aa,화서문지구대,수원중부경찰서
3,소음,20200101,012608',20200101013131.0,다사57aa17ba,세류지구대,수원남부경찰서
5,소음,20200101,014938',20200101015628.0,다사56bb18ab,유천파출소,수원서부경찰서
6,분실습득,20200101,005000',20200101005208.0,다사58ba19ab,인계파출소,수원남부경찰서


In [110]:
df_rep['rec_date_time'] = (df_rep['rec_date'].astype(str) + df_rep['rec_time'])
df_rep['rec_date_time'] = df_rep['rec_date_time'].str.replace("'", "")
df_rep['rec_date_time'] = pd.to_datetime(df_rep['rec_date_time'])

In [111]:
df_rep['arrival_date'] = df_rep['arrival_date'].astype(str)
df_rep['arrival_date'] = df_rep['arrival_date'].apply(lambda x: x[:-2])
df_rep['arrival_date'] = pd.to_datetime(df_rep['arrival_date'])

In [112]:
df_rep.head()

,case_type,rec_date,rec_time,arrival_date,gid,name,psname,rec_date_time
1,기타_타기관,20200101,012615',2020-01-01 01:41:35,다사58ab18ba,인계파출소,수원남부경찰서,2020-01-01 01:26:15
2,폭력,20200101,005151',2020-01-01 00:53:14,다사56bb21aa,화서문지구대,수원중부경찰서,2020-01-01 00:51:51
3,소음,20200101,012608',2020-01-01 01:31:31,다사57aa17ba,세류지구대,수원남부경찰서,2020-01-01 01:26:08
5,소음,20200101,014938',2020-01-01 01:56:28,다사56bb18ab,유천파출소,수원서부경찰서,2020-01-01 01:49:38
6,분실습득,20200101,005000',2020-01-01 00:52:08,다사58ba19ab,인계파출소,수원남부경찰서,2020-01-01 00:50:00


In [113]:
df_rep.drop(columns=['rec_date', 'rec_time'], axis=1, inplace=True)

In [114]:
df_rep.rename(columns={"arrival_date": "arrival_date_time"}, inplace=True)
df_rep = df_rep[['case_type', 'rec_date_time', 'arrival_date_time', 'gid', 'name', 'psname']]
df_rep.head()

,case_type,rec_date_time,arrival_date_time,gid,name,psname
1,기타_타기관,2020-01-01 01:26:15,2020-01-01 01:41:35,다사58ab18ba,인계파출소,수원남부경찰서
2,폭력,2020-01-01 00:51:51,2020-01-01 00:53:14,다사56bb21aa,화서문지구대,수원중부경찰서
3,소음,2020-01-01 01:26:08,2020-01-01 01:31:31,다사57aa17ba,세류지구대,수원남부경찰서
5,소음,2020-01-01 01:49:38,2020-01-01 01:56:28,다사56bb18ab,유천파출소,수원서부경찰서
6,분실습득,2020-01-01 00:50:00,2020-01-01 00:52:08,다사58ba19ab,인계파출소,수원남부경찰서


In [115]:
df_rep['elapsed_time'] = df_rep['arrival_date_time'] - df_rep['rec_date_time']

In [116]:
df_rep.head()

,case_type,rec_date_time,arrival_date_time,gid,name,psname,elapsed_time
1,기타_타기관,2020-01-01 01:26:15,2020-01-01 01:41:35,다사58ab18ba,인계파출소,수원남부경찰서,0 days 00:15:20
2,폭력,2020-01-01 00:51:51,2020-01-01 00:53:14,다사56bb21aa,화서문지구대,수원중부경찰서,0 days 00:01:23
3,소음,2020-01-01 01:26:08,2020-01-01 01:31:31,다사57aa17ba,세류지구대,수원남부경찰서,0 days 00:05:23
5,소음,2020-01-01 01:49:38,2020-01-01 01:56:28,다사56bb18ab,유천파출소,수원서부경찰서,0 days 00:06:50
6,분실습득,2020-01-01 00:50:00,2020-01-01 00:52:08,다사58ba19ab,인계파출소,수원남부경찰서,0 days 00:02:08


In [121]:
list_worst_df = [df_rep_00,
                 df_rep_01,
                 df_rep_02,
                 df_rep_03,
                 df_rep_04,
                 df_rep_05,
                 df_rep_06,
                 df_rep_07,
                 df_rep_08,
                 df_rep_09,
                 df_rep_10,
                 df_rep_11,
                 df_rep_12,
                 df_rep_13,
                 df_rep_14,
                 df_rep_15,
                 df_rep_16,
                 df_rep_17,
                 df_rep_18,
                 df_rep_19,
                 df_rep_20,
                 df_rep_21,
                 df_rep_22,
                 df_rep_23,
                 df_rep_24,
                 df_rep_25,
                 df_rep_26,
                 df_rep_27,
                 df_rep_28,
                 df_rep_29,
                 df_rep_30,
                 df_rep_31,
                 df_rep_32,
                 df_rep_33,
                 df_rep_34,
                 df_rep_35,
                 df_rep_36,
                 df_rep_37,
                 df_rep_38
                ]

In [137]:
elapsed_time = []
gid = []
for i in range(len(list_worst_df)):
    gid.append(list_worst_df[i]['gid'][i])
    elapsed_time.append(list_worst_df[i]['elapsed_time'].mean())
    print(f"심각 지역 {i}의 경우, 평균적으로 접수 후 {list_worst_df[i]['elapsed_time'].mean()}시간 안에 도착했습니다.")

심각 지역 0의 경우, 평균적으로 접수 후 0 days 00:06:18.255434782시간 안에 도착했습니다.
심각 지역 1의 경우, 평균적으로 접수 후 0 days 00:06:03.627118644시간 안에 도착했습니다.
심각 지역 2의 경우, 평균적으로 접수 후 0 days 00:07:58.908805031시간 안에 도착했습니다.
심각 지역 3의 경우, 평균적으로 접수 후 0 days 00:05:31.125000시간 안에 도착했습니다.
심각 지역 4의 경우, 평균적으로 접수 후 0 days 00:05:37.043859649시간 안에 도착했습니다.
심각 지역 5의 경우, 평균적으로 접수 후 0 days 00:05:54.790849673시간 안에 도착했습니다.
심각 지역 6의 경우, 평균적으로 접수 후 0 days 00:03:47.152671755시간 안에 도착했습니다.
심각 지역 7의 경우, 평균적으로 접수 후 0 days 00:04:14.157635467시간 안에 도착했습니다.
심각 지역 8의 경우, 평균적으로 접수 후 0 days 00:05:09.166666666시간 안에 도착했습니다.
심각 지역 9의 경우, 평균적으로 접수 후 0 days 00:05:02.971962616시간 안에 도착했습니다.
심각 지역 10의 경우, 평균적으로 접수 후 0 days 00:05:35.554744525시간 안에 도착했습니다.
심각 지역 11의 경우, 평균적으로 접수 후 0 days 00:06:39.928104575시간 안에 도착했습니다.
심각 지역 12의 경우, 평균적으로 접수 후 0 days 00:05:49.305084745시간 안에 도착했습니다.
심각 지역 13의 경우, 평균적으로 접수 후 0 days 00:05:54.400000시간 안에 도착했습니다.
심각 지역 14의 경우, 평균적으로 접수 후 0 days 00:04:09.897769516시간 안에 도착했습니다.
심각 지역 15의 경우, 평균적으로 접수 후 0 days 00:04:37.304474708시간 안에 

In [140]:
df_rep['elapsed_time'].mean()

Timedelta('0 days 00:04:36.549559957')

In [138]:
df_worst_arrival = pd.DataFrame()

In [139]:
df_worst_arrival['gid'] = gid
df_worst_arrival['elapsed_time'] = elapsed_time
df_worst_arrival

,gid,elapsed_time
0,다사50ab19ab,0 days 00:06:18.255434782
1,다사51ab19aa,0 days 00:06:03.627118644
2,다사52aa17bb,0 days 00:07:58.908805031
3,다사55ab23bb,0 days 00:05:31.125000
4,다사56ab22bb,0 days 00:05:37.043859649
5,다사56ab23aa,0 days 00:05:54.790849673
6,다사56ba23aa,0 days 00:03:47.152671755
7,다사56bb16ab,0 days 00:04:14.157635467
8,다사56bb20ab,0 days 00:05:09.166666666
9,다사57ba18bb,0 days 00:05:02.971962616


In [141]:
df_geo = gpd.read_file('32.수원시_표준격자(250M_x_250M).geojson')
df_geo

,gid,geometry
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ..."
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ..."
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ..."
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ..."
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ..."
...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ..."
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ..."
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ..."
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ..."


In [142]:
geometry = []
for i in range(df_geo.shape[0]):
    if df_geo.iloc[i]['gid'] in df_worst_arrival['gid'].tolist():
        geometry.append(df_geo.iloc[i]['geometry'])

geometry

In [143]:
len(geometry)

39

In [144]:
df_worst_arrival['geometry'] = geometry

In [145]:
df_worst_arrival.head()

,gid,elapsed_time,geometry
0,다사50ab19ab,0 days 00:06:18.255434782,(POLYGON ((126.9388668024126 37.27083339530338...
1,다사51ab19aa,0 days 00:06:03.627118644,(POLYGON ((126.9501617690346 37.26863291747834...
2,다사52aa17bb,0 days 00:07:58.908805031,"(POLYGON ((126.958701116179 37.2574048695601, ..."
3,다사55ab23bb,0 days 00:05:31.125000,(POLYGON ((126.9949889452437 37.31164862064046...
4,다사56ab22bb,0 days 00:05:37.043859649,(POLYGON ((127.0063326534918 37.30268261472589...


In [146]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link(df, title = "Download CSV file", filename = "worst_elapsed.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_worst_arrival)

### 2. 안전지수 '심각' 지역 내 가장 많은 범죄 유형 찾기

In [186]:
max_case_type = []
for i in range(len(list_worst_df)):
    max_case_type.append(list_worst_df[i]['case_type'].value_counts().reset_index().rename(columns={'index': 'case_type', 'case_type': 'count'})['case_type'][0])
    print(f"심각 지역 {i}의 경우, 가장 많이 발생한 범죄 유형이 {list_worst_df[i]['case_type'].value_counts().reset_index().rename(columns={'index': 'case_type', 'case_type': 'count'})['case_type'][0]}입니다")

심각 지역 0의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 1의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 2의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 3의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 4의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 5의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 6의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 7의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 8의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 9의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 10의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 11의 경우, 가장 많이 발생한 범죄 유형이 소음입니다
심각 지역 12의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 13의 경우, 가장 많이 발생한 범죄 유형이 위험방지입니다
심각 지역 14의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 15의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 16의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 17의 경우, 가장 많이 발생한 범죄 유형이 교통사고입니다
심각 지역 18의 경우, 가장 많이 발생한 범죄 유형이 행패소란입니다
심각 지역 19의 경우, 가장 많이 발생한 범죄 유형이 상담문의입니다
심각 지역 20의 경우, 가장 많이 발생한 범죄 유형이 교통사고입니다
심각 지역 21의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 22의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 23의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 24의 경우, 가장 많이 발생한 범죄 유형이 기타형사범입니다
심각 지역 25의 경우, 가장 많이 발생한 

In [187]:
max_case_type

['위험방지',
 '위험방지',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '위험방지',
 '위험방지',
 '위험방지',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '소음',
 '위험방지',
 '위험방지',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '교통사고',
 '행패소란',
 '상담문의',
 '교통사고',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '위험방지',
 '위험방지',
 '기타형사범',
 '위험방지',
 '위험방지',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '기타형사범',
 '가정폭력',
 '위험방지',
 '가정폭력',
 '위험방지']

In [189]:
df_worst_arrival['max_case_type'] = max_case_type

In [190]:
df_worst_arrival

,gid,elapsed_time,geometry,max_case_type
0,다사50ab19ab,0 days 00:06:18.255434782,(POLYGON ((126.9388668024126 37.27083339530338...,위험방지
1,다사51ab19aa,0 days 00:06:03.627118644,(POLYGON ((126.9501617690346 37.26863291747834...,위험방지
2,다사52aa17bb,0 days 00:07:58.908805031,"(POLYGON ((126.958701116179 37.2574048695601, ...",기타형사범
3,다사55ab23bb,0 days 00:05:31.125000,(POLYGON ((126.9949889452437 37.31164862064046...,기타형사범
4,다사56ab22bb,0 days 00:05:37.043859649,(POLYGON ((127.0063326534918 37.30268261472589...,기타형사범
5,다사56ab23aa,0 days 00:05:54.790849673,(POLYGON ((127.0063179229269 37.30493602249138...,위험방지
6,다사56ba23aa,0 days 00:03:47.152671755,(POLYGON ((127.0091388685413 37.30494775649449...,위험방지
7,다사56bb16ab,0 days 00:04:14.157635467,"(POLYGON ((127.012352427917 37.24411705881436,...",위험방지
8,다사56bb20ab,0 days 00:05:09.166666666,(POLYGON ((127.0121199112968 37.28017186940066...,기타형사범
9,다사57ba18bb,0 days 00:05:02.971962616,(POLYGON ((127.0206657164189 37.26668589058893...,기타형사범


In [194]:
df_worst_arrival.sort_values(['elapsed_time'], ascending=False)

,gid,elapsed_time,geometry,max_case_type
2,다사52aa17bb,0 days 00:07:58.908805031,"(POLYGON ((126.958701116179 37.2574048695601, ...",기타형사범
34,다사60bb20ab,0 days 00:06:52.917431192,(POLYGON ((127.0572404764132 37.28034923512217...,기타형사범
11,다사57ba22aa,0 days 00:06:39.928104575,(POLYGON ((127.0204798966334 37.29598036508088...,소음
28,다사59ba16aa,0 days 00:06:30.127272727,(POLYGON ((127.0433715871909 37.24198724155773...,기타형사범
36,다사61aa16bb,0 days 00:06:25.125000,(POLYGON ((127.0602440068982 37.24881158908827...,위험방지
25,다사59aa19aa,0 days 00:06:23.972972972,(POLYGON ((127.0375691046271 37.26900660788227...,기타형사범
0,다사50ab19ab,0 days 00:06:18.255434782,(POLYGON ((126.9388668024126 37.27083339530338...,위험방지
35,다사60bb20ba,0 days 00:06:12.842105263,(POLYGON ((127.0572272755207 37.28260266781783...,가정폭력
33,다사60bb16ba,0 days 00:06:10.579787234,"(POLYGON ((127.057438313612 37.24654764102352,...",기타형사범
1,다사51ab19aa,0 days 00:06:03.627118644,(POLYGON ((126.9501617690346 37.26863291747834...,위험방지


In [195]:
df_worst_arrival

,gid,elapsed_time,geometry,max_case_type
0,다사50ab19ab,0 days 00:06:18.255434782,(POLYGON ((126.9388668024126 37.27083339530338...,위험방지
1,다사51ab19aa,0 days 00:06:03.627118644,(POLYGON ((126.9501617690346 37.26863291747834...,위험방지
2,다사52aa17bb,0 days 00:07:58.908805031,"(POLYGON ((126.958701116179 37.2574048695601, ...",기타형사범
3,다사55ab23bb,0 days 00:05:31.125000,(POLYGON ((126.9949889452437 37.31164862064046...,기타형사범
4,다사56ab22bb,0 days 00:05:37.043859649,(POLYGON ((127.0063326534918 37.30268261472589...,기타형사범
5,다사56ab23aa,0 days 00:05:54.790849673,(POLYGON ((127.0063179229269 37.30493602249138...,위험방지
6,다사56ba23aa,0 days 00:03:47.152671755,(POLYGON ((127.0091388685413 37.30494775649449...,위험방지
7,다사56bb16ab,0 days 00:04:14.157635467,"(POLYGON ((127.012352427917 37.24411705881436,...",위험방지
8,다사56bb20ab,0 days 00:05:09.166666666,(POLYGON ((127.0121199112968 37.28017186940066...,기타형사범
9,다사57ba18bb,0 days 00:05:02.971962616,(POLYGON ((127.0206657164189 37.26668589058893...,기타형사범


In [192]:
df_worst_arrival['max_case_type'].nunique()

7

In [193]:
df_worst_arrival['max_case_type'].unique()

array(['위험방지', '기타형사범', '소음', '교통사고', '행패소란', '상담문의', '가정폭력'],
      dtype=object)

In [191]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link(df, title = "Download CSV file", filename = "worst__.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_worst_arrival)